<a href="https://colab.research.google.com/github/Ishank2301/Comprehensive-ML-Handbook/blob/main/artificial_neural_network_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [63]:
import numpy as np
import pandas as pd

## Part 1 - Data Preprocessing

### Importing the dataset

In [64]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [65]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [66]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [67]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [68]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [69]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [70]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [72]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ChurnPredictor(nn.Module):
  def __init__(self):
    super(ChurnPredictor, self).__init__()
    self.fc1 = nn.Linear(12,16)        # 8 features , 16 neurons in first hidden layer
    self.fc2 = nn.Linear(16,16)       # 16 neurons in second hidden layer
    self.output = nn.Linear(16,1)     # Output layer



# Forward prop:
  def forward(self,X):
    X= F.relu(self.fc1(X))
    X= F.relu(self.fc2(X))
    X = torch.sigmoid(self.output(X))
    return X

## Part 3 - Training the ANN

### Compiling the ANN

In [74]:
model = ChurnPredictor()
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    # Convert arrays to tensors
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.float32)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.6117
Epoch [20/100], Loss: 0.5884
Epoch [30/100], Loss: 0.5638
Epoch [40/100], Loss: 0.5392
Epoch [50/100], Loss: 0.5166
Epoch [60/100], Loss: 0.4980
Epoch [70/100], Loss: 0.4843
Epoch [80/100], Loss: 0.4748
Epoch [90/100], Loss: 0.4680
Epoch [100/100], Loss: 0.4626


### Evaluating the model:

In [76]:
with torch.no_grad():
  y_pred = model(torch.tensor(X_test,dtype=torch.float32))
  y_pred_cls = y_pred.round()
  acc = y_pred_cls.eq(torch.tensor(y_test).unsqueeze(1)).sum()/ float(y_test.shape[0])
  print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7975
